## 3.3 线性回归的简洁实现

In [2]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

### 3.3.1 生成数据集

In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 3.3.2 读取数据集

In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个pytorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))   # 返回一个batch的数据

[tensor([[-0.4321,  0.6545],
         [ 0.4275, -0.3939],
         [-0.3805,  0.6825],
         [-0.3136, -0.9929],
         [ 0.0290, -0.2655],
         [-0.0525,  0.1743],
         [ 1.2738, -1.2306],
         [-1.0728, -0.9019],
         [-0.2718,  0.1828],
         [-0.9911,  0.7730]]),
 tensor([[ 1.1097],
         [ 6.3750],
         [ 1.1199],
         [ 6.9444],
         [ 5.1470],
         [ 3.4892],
         [10.9130],
         [ 5.1202],
         [ 3.0407],
         [-0.4195]])]

### 3.3.3 定义模型

In [6]:
#nn是神经网络的缩写
from torch import nn

net=nn.Sequential(nn.Linear(2,1))  # 定义一个线性回归模型

### 3.3.4 初始化模型参数

In [7]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 3.3.5 定义损失函数

In [8]:
loss = nn.MSELoss()  # 均方误差损失函数

### 3.3.6 定义优化算法

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)  # 小批量随机梯度下降

### 3.3.7 训练模型

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)  # X和y的形状必须与net的输入形状一致
        trainer.zero_grad()   # 梯度清零
        l.backward()         # 反向传播，计算关于net参数的梯度
        trainer.step()        # 更新net的参数
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000240
epoch 2, loss 0.000099
epoch 3, loss 0.000100


### 3.3.8 评价模型

In [11]:
w=net[0].weight.data
b=net[0].bias.data 
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([-0.0011, -0.0006])
b的估计误差: tensor([0.0002])
